In [35]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import pickle
from itertools import product
from collections import defaultdict
from sqlalchemy import create_engine
from sklearn.feature_selection import RFE, chi2
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import sys
sys.path.append("../model")
from build_model import get_data, CancellationModel
from pipeline import Pipeline
import constants as C

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
engine = create_engine(C.ENGINE)
df = get_data(engine, booked=True)

In [67]:
with open('../model/model.pkl', 'rb') as f:
    model = pickle.load(f)

In [68]:
df, X = model.pipeline.transform(df)

In [78]:
df.head()

,user_id,created_as_guest,local_rental,awards_referral_bonus,pickup,dropoff,created_at,promo_code_id,booking_application,reservation_frequency,...,pickup_dow,weekend_pickup,winter_pickup,used_promo,used_referral,credit_card,web_booking,western_pickup,modified_profile,insurance
0,20039.0,1,0,0,2014-01-01 00:00:00.000,2014-01-09 00:00:00.000,2013-12-18 19:38:21.984000000,36977.0,web,0,...,2,0,1,1,0,0,1,0,False,Corporate
1,16089.0,0,0,0,2014-01-01 00:00:00.000,2014-01-03 13:59:59.712,2013-12-31 23:42:49.248000000,0.0,web,Repeat,...,2,0,1,0,1,1,1,0,True,Corporate
2,18397.0,0,0,0,2014-01-01 01:00:00.288,2014-01-03 23:30:00.288,2013-12-10 21:51:36.576000000,25264.0,android,New,...,2,0,1,1,1,1,0,1,True,Corporate
3,21866.0,1,0,0,2014-01-01 01:00:00.288,2014-01-01 16:00:00.288,2014-01-01 00:45:44.063999999,0.0,web,0,...,2,0,1,0,0,0,1,0,False,Corporate
4,18464.0,1,0,0,2014-01-01 01:59:59.712,2014-01-02 01:59:59.712,2013-12-11 12:37:27.264000000,36432.0,web,0,...,2,0,1,1,1,0,1,1,False,Corporate


In [72]:
df["insurance"] = df[["insurance_corporate", "insurance_personal", "insurance_silvercar"]].idxmax(axis=1)

In [73]:
df["insurance"] = df["insurance"].map({None: "None", "insurance_personal": "Personal",
                     "insurance_corporate": "Corporate", "insurance_silvercar": "Silvercar"})